In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import sys
import scipy.stats as stats
import matplotlib.pylab as plt
%matplotlib inline

In [24]:
os.getcwd()
os.listdir()

['.Rhistory',
 'js1_bi.csv',
 'BreakPointDistance.csv',
 'Bipartite_graph.ipynb',
 'Junction_Sequences_NTRK1_nonCRC_4_19_20_deid.xlsx',
 '.DS_Store',
 'Table1.rds',
 'nonCRC_Table1.pkl',
 'Fusion_Gene_ NTRK1_partners.ipynb',
 'Fusion_Gene_nonCRC.ipynb',
 'Junction_Sequences_CRC_4_19_20_deid.xlsx',
 'js1_count.pkl',
 'Bed_files_for_USCS_plot.ipynb',
 '.RData',
 'Table1.pkl',
 '.ipynb_checkpoints',
 'Fusion_Gene_R.ipynb',
 'Fusion_Gene_CRC.ipynb',
 'Fusion_Gene_R.R',
 'CeCe_Proj']

## CRC Gene1

In [2]:
with open('Table1.pkl', 'rb') as f: # load pickle
    df = pickle.load(f)
    f.close()
df = df[df.MSIstatus != 'MSI ambiguous'] # remove 'MSI ambiguous'
#df.set_index('deidentifiedSpecimenName', inplace = True)
df.head(3)

,deidentifiedSpecimenName,diseaseOntology,MSIstatus,gene1_RE,gene2_RE,Gene1_Breakpoint1,Gene2_Breakpoint1,Junction Sequence 1,Gene1_Breakpoint2,Gene2_Breakpoint2,Junction Sequence 2
0,XRN:6SJTT7,colon adenocarcinoma (crc),MSI-H,RET,TRIM24,chr10:43611914,chr7:138268067,AAGAAAAAAATCTAGATGATTAAGAGGAAATATGTGAAGACAAGTT...,-,-,-
1,XRN:DNF6WD,colon adenocarcinoma (crc),MSS,BRAF,TRIM24,chr7:140489429,chr7:138253092,AACTTTCCATTCTTGAGAGCTTACAATCTCATGGGAAGTAAGATGC...,chr7:140489430,chr7:138253093,AAAAAAAAAAAAAAAAAAGGAGGCCAGGTACGGTGGCTTACGAGTT...
4,XRN:AFAEVN,colon adenocarcinoma (crc),MSI-H,NTRK1,TPM3,chr1:156843346,chr1:154133546,GACCTGTTTCTCCCAGGCCTGCCCTTTGATTTCGGGTTCTACTCGC...,chr1:156843348,chr1:154133545,TACTTTCCAGTTGAGCATCCCAAATTCGAAAATCAGAAATTTATAC...


In [72]:
sub_df = df.drop(columns = ["diseaseOntology","Junction Sequence 1","Junction Sequence 2"]) # drop sequence
sub_df["JS1"] = sub_df[['Gene1_Breakpoint1', 'Gene2_Breakpoint1']].agg('~'.join, axis=1)
sub_df["JS2"] = sub_df[['Gene1_Breakpoint2', 'Gene2_Breakpoint2']].agg('~'.join, axis=1)
sub_df = sub_df.drop(columns = ['Gene1_Breakpoint1', 'Gene2_Breakpoint1','Gene1_Breakpoint2', 'Gene2_Breakpoint2']) 
sub_df.head(3)

,deidentifiedSpecimenName,MSIstatus,gene1_RE,gene2_RE,JS1,JS2
0,XRN:6SJTT7,MSI-H,RET,TRIM24,chr10:43611914~chr7:138268067,-~-
1,XRN:DNF6WD,MSS,BRAF,TRIM24,chr7:140489429~chr7:138253092,chr7:140489430~chr7:138253093
4,XRN:AFAEVN,MSI-H,NTRK1,TPM3,chr1:156843346~chr1:154133546,chr1:156843348~chr1:154133545


In [73]:
# Gather(melt) BP1 and BP2
# https://support.sisense.com/hc/en-us/community/posts/360037984913-Pivoting-and-Melting-Dataframes
dt = pd.melt(sub_df, id_vars =['deidentifiedSpecimenName','MSIstatus','gene1_RE','gene2_RE'],\
             value_vars =['JS1', 'JS2']).rename(columns={'variable': 'JS'})
dt = dt[dt["value"]!= "-~-" ].reset_index(drop=True) # remove empty JS2
dt.tail()

,deidentifiedSpecimenName,MSIstatus,gene1_RE,gene2_RE,JS,value
330,XRN:VGD7DC,MSS,BRAF,TRIM24,JS2,chr7:140494046~chr7:138242070
331,XRN:3GJLL5,MSS,EGFR,2020-09-14 00:00:00,JS2,chr7:55269676~chr7:55896775
332,XRN:NQJ1G5,MSS,EGFR,2020-09-14 00:00:00,JS2,chr7:55269676~chr7:55870573
333,XRN:QU97T3,MSS,RET,NCOA4,JS2,chr10:43610259~chr10:51585556
334,XRN:CFTDTX,MSS,BRAF,TMEM44,JS2,chr7:140482546~chr3:194325733


In [74]:
# Split - Vectorized string functions for Series and Index
dt[['G1','G2']] = dt.value.str.split('~', expand=True)
dt[['G1chr','G1bp']] = dt.G1.str.split(':', expand=True)
dt[['G2chr','G2bp']] = dt.G2.str.split(':', expand=True)
dt = dt.drop(columns = ['G1','G2','value'])
dt.tail()

,deidentifiedSpecimenName,MSIstatus,gene1_RE,gene2_RE,JS,G1chr,G1bp,G2chr,G2bp
330,XRN:VGD7DC,MSS,BRAF,TRIM24,JS2,chr7,140494046,chr7,138242070
331,XRN:3GJLL5,MSS,EGFR,2020-09-14 00:00:00,JS2,chr7,55269676,chr7,55896775
332,XRN:NQJ1G5,MSS,EGFR,2020-09-14 00:00:00,JS2,chr7,55269676,chr7,55870573
333,XRN:QU97T3,MSS,RET,NCOA4,JS2,chr10,43610259,chr10,51585556
334,XRN:CFTDTX,MSS,BRAF,TMEM44,JS2,chr7,140482546,chr3,194325733


In [75]:
dg = dt.copy()
dg.head()

,deidentifiedSpecimenName,MSIstatus,gene1_RE,gene2_RE,JS,G1chr,G1bp,G2chr,G2bp
0,XRN:6SJTT7,MSI-H,RET,TRIM24,JS1,chr10,43611914,chr7,138268067
1,XRN:DNF6WD,MSS,BRAF,TRIM24,JS1,chr7,140489429,chr7,138253092
2,XRN:AFAEVN,MSI-H,NTRK1,TPM3,JS1,chr1,156843346,chr1,154133546
3,XRN:MPPMPA,MSI-H,NTRK1,TPM3,JS1,chr1,156843424,chr1,154133209
4,XRN:30NP36,MSI-H,NTRK1,TPM3,JS1,chr1,156843471,chr1,154132639


In [76]:
gene_col="gene1_RE"
dg[gene_col].unique()

array(['RET', 'BRAF', 'NTRK1', 'FGFR2', 'ETV6', 'NTRK3', 'ALK', 'RAF1',
       'FGFR3', 'ROS1', 'MET', 'EGFR', 'FGFR1', 'NTRK2'], dtype=object)

In [77]:
"_".join(["ab"]) == "_".join(["a","b"])

False

In [83]:
Chr_col = "G1chr"
BP_col = "G1bp"
# dt = pd.DataFrame()
# dt["chrom"] = dg[Chr_col]
# dt["start"] = dg[BP_col].astype(int)
# dt["end"] = dt["start"] + 1
# dt.head()
bed_coulmns(dg, Chr_col,BP_col).head()
#dt.shape

,chrom,start,end
0,chr10,43611914,43611915
1,chr7,140489429,140489430
2,chr1,156843346,156843347
3,chr1,156843424,156843425
4,chr1,156843471,156843472


In [27]:
# Try save DataFrame into a functional bed file
# os.getcwd()
# dt.to_csv("tmp.bed", sep='\t', header=False, index=False)
save_as_bed(dt,save_path=".", save_name="tmp1.bed")

In [3]:
# select BED3 columns
def bed_coulmns(df, Chr_col,BP_col):
    dt = pd.DataFrame()
    dt["chrom"] = df[Chr_col]
    dt["start"] = df[BP_col].astype(int)
    dt["end"] = dt["start"] + 1
    return dt

# convert DataFrame to bed file and save
def save_as_bed(df,save_path=".", save_name="tmp.bed"):
    path_name = os.path.join(save_path, save_name)
    df.to_csv(path_name, sep='\t', header=False, index=False)
    return


def sub_by_MSI(df,MSI_col):
    dt = df.copy()
    dt_msi = dt[dt[MSI_col]=="MSI-H"]
    dt_mss = dt[dt[MSI_col]=="MSS"]
    return dt_msi, dt_mss


# split df by genes and save bed file for mss and msi separately
def bed_subtable_save(df,MSI_col,gene_col,Chr_col,BP_col,save_path):
    if not os.path.exists(save_path): os.mkdir(save_path)
    genes = df[gene_col].unique()
    for i in genes:
        gene_path = os.path.join(save_path, i)
        if not os.path.exists(gene_path): os.mkdir(gene_path)
        msi_name = i+"_msi.bed"
        mss_name = i+"_mss.bed"
        
        sub_df = df[df[gene_col] == i]
        msi, mss = sub_by_MSI(sub_df, MSI_col)
        msi = bed_coulmns(msi, Chr_col,BP_col)
        mss = bed_coulmns(mss, Chr_col,BP_col)
        
        #print(os.path.join(gene_path, mss_name))
        if len(msi) !=0:
            save_as_bed(msi,gene_path,msi_name)
        if len(mss) !=0:
            save_as_bed(mss,gene_path,mss_name)   
    return

In [95]:
bed_subtable_save(dg,MSI_col="MSIstatus",gene_col="gene1_RE",Chr_col="G1chr",BP_col="G1bp",save_path="CRC_gene1_bed")

## CRC Gene2

In [96]:
bed_subtable_save(dg,MSI_col="MSIstatus",gene_col="gene2_RE",Chr_col="G2chr",BP_col="G2bp",save_path="CRC_gene2_bed")

FileNotFoundError: [Errno 2] No such file or directory: 'CRC_gene2_bed/DAAM1/HIPK2'

In [ ]:
# Save path as pickle
with open(os.path.join(Fea_path,FeaXMap_pkl), 'wb') as f: #Save paths to Fea intermediates:
    pickle.dump(FeaXMap_list, f)
    f.close()
    
    # get path from pickle
def Get_Feas(Fea_path, Fea_pkl):
    with open(os.path.join(Fea_path, Fea_pkl), 'rb') as f:
        tmp = pickle.load(f)
        f.close()
    return tmp
def Get_Maps(Map_path, Map_pkl):
    with open(os.path.join(Map_path, Map_pkl), 'rb') as f:
        tmp = pickle.load(f)
        f.close()
    return tmp

## nonCRC gene1 (NTRK1 only)

In [5]:
with open('nonCRC_Table1.pkl', 'rb') as f: # load pickle
    nonCRC_df = pickle.load(f)
    f.close()
nonCRC_df = nonCRC_df[nonCRC_df.MSIstatus != 'MSI ambiguous'] # remove 'MSI ambiguous'
nonCRC_df.head()

,deidentifiedSpecimenName,diseaseOntology,MSIstatus,gene1_RE,gene2_RE,Gene1_Breakpoint1,Gene2_Breakpoint1,Junction Sequence 1,Gene1_Breakpoint2,Gene2_Breakpoint2,Junction Sequence 2
0,XRN:5H19TB,breast carcinoma (nos),MSS,NTRK1,LMNA,chr1:156844991,chr1:156109619,GGGAATGAGGTGGGAGGTGGAAGAAGGGAGAAGAAAGGTGAGTTTG...,-,-,-
1,XRN:67DD1N,brain glioblastoma (gbm),MSS,NTRK1,BCAN,chr1:156844548,chr1:156629350,TATGAGCCCAGGCAAGTTGGATGCTTCTGTGTGTATGTCCGTGACA...,-,-,-
2,XRN:8PFVJK,lung adenocarcinoma,MSS,NTRK1,TFG,chr1:156843796,chr3:100450467,TTACACTATACACATTTCCGCCCCTCCTGTTACCCAGAGTGGAGTA...,-,-,-
3,XRN:2AUNEA,breast carcinoma (nos),MSS,NTRK1,TPM3,chr1:156844354,chr1:154139659,TACTTTTTCTTCTTTAAACCTTTCTTACATATTTGAGACTTGCTAC...,-,-,-
4,XRN:L6E0XT,unknown primary adenocarcinoma,MSI-H,NTRK1,LMNA,chr1:156845528,chr1:156109315,CAAGGGCAGGGACGAGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTG...,-,-,-


In [8]:
def clean_df(df):
    tmp = df.drop(columns = ["diseaseOntology","Junction Sequence 1","Junction Sequence 2"]) # drop sequence
    tmp["JS1"] = tmp[['Gene1_Breakpoint1', 'Gene2_Breakpoint1']].agg('~'.join, axis=1)
    tmp["JS2"] = tmp[['Gene1_Breakpoint2', 'Gene2_Breakpoint2']].agg('~'.join, axis=1)
    tmp = tmp.drop(columns = ['Gene1_Breakpoint1', 'Gene2_Breakpoint1','Gene1_Breakpoint2', 'Gene2_Breakpoint2']) 

    dt = pd.melt(tmp, id_vars =['deidentifiedSpecimenName','MSIstatus','gene1_RE','gene2_RE'],\
                 value_vars =['JS1', 'JS2']).rename(columns={'variable': 'JS'})
    dt = dt[dt["value"]!= "-~-" ].reset_index(drop=True) # remove empty JS2
    dt[['G1','G2']] = dt.value.str.split('~', expand=True)
    dt[['G1chr','G1bp']] = dt.G1.str.split(':', expand=True)
    dt[['G2chr','G2bp']] = dt.G2.str.split(':', expand=True)
    dt = dt.drop(columns = ['G1','G2','value'])
    return dt

In [9]:
dg = clean_df(nonCRC_df)
bed_subtable_save(dg,MSI_col="MSIstatus",gene_col="gene1_RE",Chr_col="G1chr",BP_col="G1bp",save_path="nonCRC_gene1_bed")